распознаем автомобили и выделим цвета 2мя способами: средний цвет в области автомобиля и доминирующий цвет в области атомобиля

In [2]:
!pip install opencv-python-headless matplotlib scikit-learn
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg -O yolov4-tiny.cfg
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights -O yolov4-tiny.weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/data/coco.names -O coco.names

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import CSS4_COLORS
from matplotlib.patches import Rectangle
from IPython.display import display, Markdown
from collections import Counter
from sklearn.cluster import KMeans
from google.colab import drive


with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

net = cv2.dnn.readNet("yolov4-tiny.weights", "yolov4-tiny.cfg")
layer_names = net.getLayerNames()

try:
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
except:
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

vehicle_classes = [2, 3, 5, 7]  # car, motorcycle, bus, truck


drive.mount('/content/drive')
IMAGE_DIR = '/content/drive/MyDrive/task2'
VALID_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp')
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(VALID_EXTENSIONS)]


def create_color_database():
    color_db = {}
    for name, hex_code in CSS4_COLORS.items():
        hex_code = hex_code.lstrip('#')
        if len(hex_code) == 6:
            r, g, b = (int(hex_code[i:i+2], 16) for i in (0, 2, 4))
            color_db[name] = (r, g, b)

    car_colors = {
        "авто-красный": (200, 30, 30),
        "авто-синий": (30, 60, 180),
        "авто-белый": (245, 245, 245),
        "авто-черный": (20, 20, 20),
        "авто-серебристый": (200, 200, 200),
        "авто-серый": (150, 150, 150),
        "авто-бежевый": (210, 180, 140),
        "авто-зеленый": (30, 120, 30),
        "авто-желтый": (255, 220, 30),
        "авто-оранжевый": (255, 150, 30),
        "авто-голубой": (100, 150, 200),
        "авто-фиолетовый": (150, 50, 200),
        "авто-коричневый": (150, 75, 0),
        "авто-бирюзовый": (0, 200, 200),
        "авто-розовый": (255, 100, 150),
    }
    color_db.update(car_colors)
    return color_db

# Поиск ближайшего названия цвета
def find_closest_color(rgb_tuple, color_db):
    r, g, b = rgb_tuple
    min_dist = float('inf')
    closest_name = 'unknown'
    for name, (cr, cg, cb) in color_db.items():
        dist = (r - cr)**2 + (g - cg)**2 + (b - cb)**2
        if dist < min_dist:
            min_dist = dist
            closest_name = name
    return closest_name

# Детекция автомобилей
def detect_cars(image, confidence_threshold=0.5):
    height, width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > confidence_threshold and class_id in vehicle_classes:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, 0.4)

    if indexes is not None and len(indexes) > 0:
        indexes = indexes.flatten()
        max_area = 0
        best_box = None
        for i in indexes:
            x, y, w, h = boxes[i]
            area = w * h
            if area > max_area:
                max_area = area
                best_box = (x, y, w, h)
        return best_box
    return None

# Функция для нахождения среднего цвета
def find_average_color(roi):
    """Находит средний цвет в области"""
    if roi.size == 0:
        return (0, 0, 0)
    avg_color = np.mean(roi, axis=(0, 1)).astype(int)
    return tuple(avg_color)

# Функция для нахождения доминирующего цвета с использованием K-средних
def find_dominant_color(roi, n_clusters=5):
    """Находит доминирующий цвет в области с помощью кластеризации K-средних"""
    if roi.size == 0:
        return (0, 0, 0)


    pixels = roi.reshape(-1, 3)


    kmeans = KMeans(n_clusters=min(n_clusters, len(pixels)), n_init=10)
    kmeans.fit(pixels)


    colors = kmeans.cluster_centers_
    counts = np.bincount(kmeans.labels_)


    valid_clusters = []
    for i, color in enumerate(colors):
        brightness = 0.299 * color[0] + 0.587 * color[1] + 0.114 * color[2]
        if brightness > 30:  # Игнорируем очень темные цвета
            valid_clusters.append((counts[i], color))


    if not valid_clusters:
        dominant_idx = np.argmax(counts)
        dominant_color = colors[dominant_idx]
    else:

        valid_clusters.sort(key=lambda x: x[0], reverse=True)
        dominant_color = valid_clusters[0][1]


    return tuple(np.round(dominant_color).astype(int))


COLOR_DB = create_color_database()
all_results = []

for img_file in image_files:
    try:
        img_path = os.path.join(IMAGE_DIR, img_file)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Ошибка загрузки: {img_file}")
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        orig_image = image_rgb.copy()
        height, width = orig_image.shape[:2]

        # Детекция автомобилей
        car_box = detect_cars(image)
        car_found = car_box is not None

        # Определение области анализа
        if car_found:
            x, y, w, h = car_box
            x, y = max(0, x), max(0, y)
            w, h = min(width - x, w), min(height - y, h)

            # Добавляем небольшой отступ от границ автомобиля
            padding = min(10, w // 10, h // 10)
            roi = orig_image[y+padding:y+h-padding, x+padding:x+w-padding]
        else:
            # Анализируем центр изображения
            roi = orig_image[height//4:3*height//4, width//4:3*width//4]

        # Находим средний цвет
        average_color = find_average_color(roi)
        avg_r, avg_g, avg_b = average_color

        # Находим доминирующий цвет
        dominant_color = find_dominant_color(roi)
        dom_r, dom_g, dom_b = dominant_color

        # Определение названий цветов
        avg_color_name = find_closest_color(average_color, COLOR_DB)
        dom_color_name = find_closest_color(dominant_color, COLOR_DB)

        # Сохранение результатов
        all_results.append({
            'filename': img_file,
            'avg_rgb': average_color,
            'avg_color_name': avg_color_name,
            'dom_rgb': dominant_color,
            'dom_color_name': dom_color_name,
            'car_found': car_found
        })

        # Визуализация отчета
        fig, ax = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle(f'Анализ: {img_file}', fontsize=18, weight='bold')

        # Исходное изображение
        ax[0].imshow(orig_image)
        if car_found:
            rect = Rectangle((x, y), w, h, linewidth=3, edgecolor='lime', facecolor='none')
            ax[0].add_patch(rect)
            title = 'Обнаружен автомобиль'
        else:
            rect = Rectangle((width//4, height//4), width//2, height//2,
                             linewidth=3, edgecolor='red', facecolor='none')
            ax[0].add_patch(rect)
            title = 'Автомобиль не обнаружен! Анализ центра'
        ax[0].set_title(title, fontsize=14)
        ax[0].axis('off')


        avg_color_block = np.zeros((200, 300, 3), dtype=np.uint8)
        avg_color_block[:, :] = average_color
        ax[1].imshow(avg_color_block)


        avg_brightness = avg_r*0.299 + avg_g*0.587 + avg_b*0.114
        avg_text_color = 'white' if avg_brightness < 128 else 'black'

        avg_text = (f"Средний цвет\n"
                   f"Название: {avg_color_name}\n"
                   f"RGB: {average_color}\n"
                   f"RG: ({avg_r}, {avg_g})")

        ax[1].text(150, 100, avg_text, fontsize=12, ha='center', va='center',
                  color=avg_text_color, bbox=dict(facecolor='black', alpha=0.3))
        ax[1].set_title('Средний цвет', fontsize=14)
        ax[1].axis('off')


        dom_color_block = np.zeros((200, 300, 3), dtype=np.uint8)
        dom_color_block[:, :] = dominant_color
        ax[2].imshow(dom_color_block)


        dom_brightness = dom_r*0.299 + dom_g*0.587 + dom_b*0.114
        dom_text_color = 'white' if dom_brightness < 128 else 'black'

        dom_text = (f"Доминирующий цвет\n"
                   f"Название: {dom_color_name}\n"
                   f"RGB: {dominant_color}\n"
                   f"RG: ({dom_r}, {dom_g})")

        ax[2].text(150, 100, dom_text, fontsize=12, ha='center', va='center',
                  color=dom_text_color, bbox=dict(facecolor='black', alpha=0.3))
        ax[2].set_title('Доминирующий цвет', fontsize=14)
        ax[2].axis('off')

        plt.tight_layout()
        plt.show()

        # Текстовый отчет
        display(Markdown(f"**Отчет для:** `{img_file}`"))
        display(Markdown(f"- Автомобиль обнаружен: **{'да' if car_found else 'нет'}**"))
        display(Markdown("### Средний цвет:"))
        display(Markdown(f"- RGB: **{average_color}**"))
        display(Markdown(f"- Название: **{avg_color_name}**"))
        display(Markdown(f"- Для оборудования: **RG({avg_r}, {avg_g})**"))
        display(Markdown("### Доминирующий цвет:"))
        display(Markdown(f"- RGB: **{dominant_color}**"))
        display(Markdown(f"- Название: **{dom_color_name}**"))
        display(Markdown(f"- Для оборудования: **RG({dom_r}, {dom_g})**"))
        display(Markdown("---"))

    except Exception as e:
        display(Markdown(f"**Ошибка:** {str(e)}"))
        display(Markdown("---"))

# Итоговый отчет
if all_results:
    display(Markdown("## Итоговая таблица результатов"))
    display(Markdown("| Файл | Авто | Средний цвет | Доминирующий цвет |"))
    display(Markdown("|------|-----|--------------|-------------------|"))

    for res in all_results:
        status = "✓" if res['car_found'] else "✗"
        display(Markdown(
            f"| {res['filename']} | {status} | {res['avg_color_name']} ({res['avg_rgb']}) | {res['dom_color_name']} ({res['dom_rgb']}) |"
        ))

    cars_found = sum(1 for res in all_results if res['car_found'])
    display(Markdown(f"**Итоги:**"))
    display(Markdown(f"- Обработано: **{len(all_results)} изображений**"))
    display(Markdown(f"- Найдено авто: **{cars_found}**"))
    display(Markdown(f"- Без авто: **{len(all_results) - cars_found}**"))
else:
    display(Markdown("**Нет данных для отображения**"))

Output hidden; open in https://colab.research.google.com to view.